In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import imageio

In [ ]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define hyperparameters
latent_dim = 100
img_size = 64
channels = 3  # Assuming RGB images
output_dir = 'Car_GAN'
epochs = 50
batch_size = 64
learning_rate = 0.0002

In [ ]:
# Define image shape
img_shape = (channels, img_size, img_size)

In [ ]:
# Define Generator model
class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(img.size(0), *img_shape)

In [ ]:
# Define Discriminator model
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        verdict = self.model(img_flat)
        return verdict

In [ ]:
def train(generator, discriminator, optimizer_generator, optimizer_discriminator, data_loader, device, latent_dim, img_shape, epochs, output_dir):
    adversarial_loss = nn.BCELoss()
    losses = []
    images_for_gif = []
    Tensor = torch.FloatTensor

    for epoch in range(1, epochs+1):
        for i, (images, _) in enumerate(data_loader):
            real_images = images.to(device)
            real_output = Variable(Tensor(images.size(0), 1).fill_(1.0), requires_grad=False)
            fake_output = Variable(Tensor(images.size(0), 1).fill_(0.0), requires_grad=False)

            optimizer_generator.zero_grad()
            z = Variable(Tensor(np.random.normal(0, 1, (images.shape[0], latent_dim))).to(device))
            generated_images = generator(z)
            generator_loss = adversarial_loss(discriminator(generated_images), real_output)
            generator_loss.backward()
            optimizer_generator.step()

            optimizer_discriminator.zero_grad()
            discriminator_loss_real = adversarial_loss(discriminator(real_images), real_output)
            discriminator_loss_fake = adversarial_loss(discriminator(generated_images.detach()), fake_output)
            discriminator_loss = (discriminator_loss_real + discriminator_loss_fake) / 2
            discriminator_loss.backward()
            optimizer_discriminator.step()

            print(f"[Epoch {epoch:=4d}/{epochs}] [Batch {i+1:=4d}/{len(data_loader)}] ---> "
                  f"[D Loss: {discriminator_loss.item():.6f}] [G Loss: {generator_loss.item():.6f}]")

        losses.append((generator_loss.item(), discriminator_loss.item()))
        image_filename = f'{output_dir}/images/{epoch}.png'
        save_image(generated_images.data[:1], image_filename, normalize=True)
        images_for_gif.append(imageio.imread(image_filename))

    # Visualizing the losses at every epoch
    losses = np.array(losses)
    plt.plot(losses.T[0], label='Generator')
    plt.plot(losses.T[1], label='Discriminator')
    plt.title("Training Losses")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.savefig(f'{output_dir}/loss_plot.png')

    # Creating a gif of generated images at every epoch
    imageio.mimwrite(f'{output_dir}/generated_images.gif', images_for_gif, fps=len(images)/5)

In [ ]:
# Initialize generator and discriminator models
generator = Generator(latent_dim, (channels, img_size, img_size)).to(device)
discriminator = Discriminator((channels, img_size, img_size)).to(device)

In [ ]:
# Initialize optimizers
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

In [ ]:
# Load dataset
data_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
dataset = datasets.ImageFolder(root="Car", transform=data_transform)  # Adjust the path
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
# Create output directory if it doesn't exist
os.makedirs(f'{output_dir}/images', exist_ok=True)

In [ ]:
# Train GAN
train(generator, discriminator, optimizer_generator, optimizer_discriminator, data_loader, device, latent_dim, img_shape, epochs, output_dir)